# Série 1 - Exercice 2

Ici on va faire une petite introduction à Diffie-Hellman. La première partie va donne une introduction dans les opérations cryptographiques nécessaires. La deuxième partie vérifie que l'opération Diffie-Hellman marche. Et la troisième partie combine le Diffie-Hellman avec la signature nécessaire pour pouvoir utiliser une clé à longue durée.

## 1. Connaissance

Création de clés: libnacl vous donne la possibilité de créer une paire de clés avec la méthode suivante:

priv, pub = libnacl.crypto_keypair()

Multiplication: avec les courbes elliptiques, une multiplications peut se faire entre deux clés privées (vu que ce sont des nombres) ou entre une clé privée et une clé publique. Si vous connaissez le RSA, ceci correspond à peu près à l'exponentiation modulaire. Avec libnacl, la méthode à utiliser est la suivante:

resultat = crypto_scalarmult(priv, pub)

Dans l'exemple suivant, on va vérifier que les deux parties du Diffie-Hellman donnent bien le même résultat.

In [ ]:
# Série 1 - Exercice 2 - Partie 1

import libnacl, ctypes

def crypto_scalarmult(scal, point):
    '''
    Cette méthode retourne le résultat de la multiplication d'un scalaire avec
    un point.
    '''
    if len(scal) != libnacl.crypto_box_SECRETKEYBYTES:
        raise ValueError('Invalid secret key')
    if len(point) != libnacl.crypto_box_PUBLICKEYBYTES:
        raise ValueError('Invalid public key')
    result = ctypes.create_string_buffer(libnacl.crypto_box_PUBLICKEYBYTES)
    if libnacl.nacl.crypto_scalarmult(result, scal, point):
        raise libnacl.CryptError('Failed to compute scalar multiplication')
    return result.raw

pub, priv = libnacl.crypto_box_keypair()

res = crypto_scalarmult(priv, pub)

print("Clé privée:", priv.hex())
print("Clé publique:", pub.hex())
print("Multiplication:", res.hex())


## 2. Compréhension

Supposons qu'Alice et Bob veulent créer une clé symétrique en utilisant Diffie-Hellman. Ils vont donc faire la chose suivante:

1. Alice et Bob vont créer leurs paires de clés:
```
alice_pub, alice_priv pour Alice
bob_pub, bob_priv pour Bob
```


2. Alice envoie sa clé publique (alice_pub) à Bob
3. Bob envoie sa clé publique (bob_pub) à Alice
3. Bob fait le calcul suivant:
```
bob_symetrique = bob_priv * alice_pub
```


4. Alice fait de même:
```
alice_symetrique = alice_priv * bob_pub
```


A la fin les deux doivent avoir le même résultat!

Utilisez les éléments que vous avez vu dans la 1ère partie pour implémenter cet algorithme! Vous n'avez pas besoin de copier la méthode `crypto_scalarmult` - elle est disponible une fois que vous avez exécuté le block correspondant.

Pour finaliser l'exercice, vous pouvez encore faire le hachage sha256 du résultat de la multiplication. Ceci assure que d'éventuels structures de la clé publique ne sont plus visibles.

In [ ]:
# Série 1 - Exercice 1 - Partie 2


## 3. Application

Maintenant on suppose qu'Alice et Bob ont une paire de clés long terme, c-à-dire qu'ils utilisent pour plusieurs échanges Diffie-Hellman. Ces clés sont connues d'avance, donc Bob connaît la clé publique longue durée d'Alice, et vice-versa.

Ajouter la signature à l'exercice sur le Diffie-Hellman, en vérifiant de chaque côté. On aura donc:

### Préparation:

Alice et Bob choisissent leur paires de clés longue durée et échangent la partie publique: alice_pub_long et bob_pub_long

### Algorithme:

1. Alice et Bob choisissent une paire de clés pour la session
2. Alice envoie sa clé publique, signée par sa clé longue durée
3. Bob vérifie la signature avec la clé publique de longue durée d'Alice
4. Bob envoie sa clé publique, signée par sa clé longue durée
5. Alice vérifie la signature avec la clé publique de longue durée de Bob
6. Alice et Bob font le calcule Diffie-Hellman, puis font un sha256 dessus

In [ ]:
# Série 1 - Exercice 1 - Partie 3